# Nombre de la asignatura: Mineria de datos
### Carrera: Big Data
### Estudiante: Paul Rodriguez,  Ariel Saquicela
### Fecha:22/04/2024

### Objetivos de la Práctica:

In [5]:
import mysql.connector
import pandas as pd

In [7]:
# Especifica la ruta de tu archivo Excel
ruta_archivo_excel = "Datos exel/Datos sin nulos.xlsx"

# Lee el archivo Excel y conviértelo a un DataFrame
df = pd.read_excel(ruta_archivo_excel)

# Muestra las primeras filas del DataFrame
df.head()

,Unnamed: 0,Nombre,Edad,Puntuacion,Sexo
0,0,Anthony Wallace,46,86,M
1,1,Steven Barker,25,67,F
2,2,Barry Graham,56,82,M
3,3,Austin Jackson,40,92,M
4,6,Zachary Sampson,20,85,M


In [9]:
#Método para conectar a la base de datos
def establecer_conexion(host, user, password, database):
    try:
        conexion = mysql.connector.connect(
            host="localhost",
            user="root",
            password="",
            database="estudiantes"
        )
        return conexion
    except mysql.connector.Error as err:
        print(f"Error: {err}")
        return None

In [11]:
conexion = establecer_conexion("localhost", "root", "", "estudiantes")
#Comprobación de la conexión
conexion.is_connected()

True

In [10]:
def limpiar_base_datos(conexion):
    try:
        cursor = conexion.cursor()
        cursor.execute("SET FOREIGN_KEY_CHECKS=0;")

        delete_queries = [
            'DELETE FROM estudiantes;',
            'DELETE FROM genero;',
            'DELETE FROM estudiantes_stg;'
        ]

        for query in delete_queries:
            cursor.execute(query)

        reset_auto_increment_queries = [
            'ALTER TABLE estudiantes AUTO_INCREMENT = 1;',
            'ALTER TABLE genero AUTO_INCREMENT = 1;',
            'ALTER TABLE estudiantes_stg AUTO_INCREMENT = 1;'
        ]

        for query in reset_auto_increment_queries:
            cursor.execute(query)

        conexion.commit()

        print("Base de datos limpiada exitosamente.")

    except Exception as e:
        print(f"Error al limpiar la base de datos: {e}")

    finally:
        cursor.execute("SET FOREIGN_KEY_CHECKS=1;")



In [12]:
# Uso de la función para limpiar la base de datos
limpiar_base_datos(conexion)


Base de datos limpiada exitosamente.


In [14]:
def insert_estudiantes_stg(conexion, df):
    cursor = conexion.cursor()

    insert_query = "INSERT INTO estudiantes_stg(nombre, edad, puntuacion, genero) VALUES (%s, %s, %s, %s)"

    try:
        for index, row in df.iterrows():
            estudiantes_data = (row['Nombre'], row['Edad'], row['Puntuacion'], row['Sexo'])
            cursor.execute(insert_query, estudiantes_data)

        conexion.commit()
        print("Datos insertados correctamente")

    except Exception as e:
        print(f"Error al insertar datos: {e}")

    finally:
        cursor.close()



In [16]:
# Uso de la función para insertar datos en la tabla estudiantes_stg
insert_estudiantes_stg(conexion, df)  # Asegúrate de tener el DataFrame `df` correctamente cargado con los datos a insertar


Datos insertados correctamente


In [19]:
def obtener_distinct_values(conexion, column_name):
    try:
        cursor = conexion.cursor()
        select_query = f"SELECT DISTINCT({column_name}) FROM estudiantes_stg"
        cursor.execute(select_query)
        distinct_values = cursor.fetchall()
        print(f"Datos extraidos correctamente de: {column_name}")
        return tuple(value[0] for value in distinct_values)
    except Exception as e:
        print(f"Error al obtener valores distintos: {e}")
        return []
    finally:
        if cursor:
            cursor.close()

In [28]:
distinct_values_genero=obtener_distinct_values(conexion, "genero")
distinct_values_genero

Datos extraidos correctamente de: genero


('M', 'F')

In [32]:
def insert_distinct_values_into_table(conexion, distinct_values, table_name, column_name):
    try:
        cursor = conexion.cursor()
        insert_query = f"INSERT INTO {table_name} ({column_name}) VALUES (%s)"
        for value in distinct_values:
            cursor.execute(insert_query, (value,))

        conexion.commit()
        print(f"Datos insertados correctamente en {table_name}.")

    except Exception as e:
        print(f"Error al insertar datos en {table_name}: {e}")
    finally:
        if cursor:
            cursor.close()


In [38]:
insert_distinct_values_into_table(conexion, distinct_values_genero, "genero", "id_genero")

Datos insertados correctamente en genero.


In [65]:
import pymysql

def obtener_valores_taxi(conexion):
    try:
        cursor = conexion.cursor()
        select_query = '''
        SELECT estudiantes_stg.nombre, estudiantes_stg.edad, estudiantes_stg.puntuacion, genero.id_genero
        FROM estudiantes_stg
        INNER JOIN genero ON estudiantes_stg.genero = id_genero;
        '''
        cursor.execute(select_query)
        values = cursor.fetchall()
        cursor.close()  # Cierra el cursor después de obtener los valores
        return tuple(values)

    except Exception as e:
        print(f"Error al obtener valores distintos: {e}")
        return []

In [67]:
valores=obtener_valores_taxi(conexion)
valores


(('Anthony Wallace', 46, '86', 'M'),
 ('Steven Barker', 25, '67', 'F'),
 ('Barry Graham', 56, '82', 'M'),
 ('Austin Jackson', 40, '92', 'M'),
 ('Zachary Sampson', 20, '85', 'M'),
 ('Heather Moore', 61, '60', 'F'),
 ('James Ross', 19, '96', 'M'),
 ('Adam Trevino', 38, '68', 'F'),
 ('Cory Day', 59, '82', 'M'),
 ('Margaret Martin', 32, '91', 'M'),
 ('Abigail Gilbert', 61, '62', 'F'),
 ('Kenneth Sanchez', 20, '83', 'F'),
 ('Matthew Weaver', 31, '73', 'F'),
 ('Chad Davis', 24, '98', 'M'),
 ('Vincent Braun', 25, '50', 'F'),
 ('Nicole Lyons', 53, '74', 'M'),
 ('Shane Douglas', 21, '65', 'F'),
 ('Charles Wilson MD', 51, '50', 'M'),
 ('Lauren Kim', 53, '71', 'M'),
 ('Richard Williams', 57, '78', 'F'),
 ('Sarah Morton MD', 62, '60', 'F'),
 ('Bradley Shepherd', 42, '89', 'F'),
 ('Anna Sosa', 25, '51', 'F'),
 ('Adam Smith', 34, '61', 'M'),
 ('Lauren Dennis', 26, '94', 'M'),
 ('Tammy Berger', 51, '62', 'M'),
 ('Aaron Lambert', 41, '78', 'M'),
 ('Marcus Deleon', 44, '52', 'F'),
 ('Kimberly Wheeler',

In [70]:
def insert_tabla_estudiantes(conexion, values):
    try:
        cursor = conexion.cursor()
        insert_query = "INSERT INTO estudiantes (nombre, edad, puntuacion, id_genero) VALUES (%s, %s, %s, %s)"

        for value in values:
            cursor.execute(insert_query, value)

        conexion.commit()
        print("Datos insertados correctamente.")

    except Exception as e:
        print(f"Error al insertar datos: {e}")
    
    finally:
        if cursor:
            cursor.close()

def cerrar_conexion(conexion):
    if conexion:
        conexion.close()

In [73]:
insert_tabla_estudiantes(conexion, valores)

Datos insertados correctamente.
